# E2E example with logging

In a real production setup a lot of other concerns need to be handled as well:

- hyper parameter optimization
- high availability
- fast serving
- authorization
- logging
- monitoring of quality over time

Here, no standard has evolved yet.

Kubernetes looks like the winner for hosting any mordern IT workload. But serving infrastructure tailored to ML use cases on top of k8s is still unclear.
Two frameworks which look interesting:
- https://www.kubeflow.org
- http://clipper.ai/

> Do not get intimidated by the ecosystem: https://landscape.cncf.io 

Have a look at them after the course.
Here, I will show you something a bit simpler but still with some of the most important components available:


- https://github.com/ThoughtWorksInc/ml-cd-starter-kit
- https://github.com/ThoughtWorksInc/ml-app-template

https://www.kubeflow.org/docs/other-guides/virtual-dev/getting-started-minikube/

## Addition

## Setup

## running it - creating a prediction